In [1]:
from ohlcv_data import OHLCVDataLoader
import numpy as np
import pandas as pd


folder_path = "/Users/elyazidbenkhadra/Downloads/stock_market_data/sp500/csv"
combined_data = OHLCVDataLoader.load_all_data(folder_path, clean=True, years=20)

2025-04-22 00:48:02,214 - WARNING - Skipping FPLPF.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:02,487 - WARNING - Skipping KGNR.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:02,487 - WARNING - Skipping KGNR.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:02,744 - WARNING - Skipping NOXL.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:02,744 - WARNING - Skipping NOXL.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:02,868 - WARNING - Skipping UEEC.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:02,868 - WARNING - Skipping UEEC.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:03,239 - WARNING - Skipping WSPOF.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48:03,239 - WARNING - Skipping WSPOF.csv: More than 10% of 'Volume' values are zero or missing.
2025-04-22 00:48

In [2]:
print(combined_data.head())

             CSCO_Low  CSCO_Open  CSCO_Volume  CSCO_High  CSCO_Close  \
Date                                                                   
2005-04-25  17.389999  17.459999     39358000  17.600000   17.480000   
2005-04-26  17.250000  17.350000     56444600  17.639999   17.280001   
2005-04-27  17.080000  17.190001     62447400  17.459999   17.250000   
2005-04-28  17.100000  17.139999     48957400  17.330000   17.139999   
2005-04-29  17.040001  17.190001     49386700  17.299999   17.270000   

            CSCO_Adjusted Close    UAL_Low  UAL_Open  UAL_Volume   UAL_High  \
Date                                                                          
2005-04-25            12.413069  33.700001     34.82   1940300.0  34.939999   
2005-04-26            12.271045  33.700001     34.82   1940300.0  34.939999   
2005-04-27            12.249736  33.700001     34.82   1940300.0  34.939999   
2005-04-28            12.171627  33.700001     34.82   1940300.0  34.939999   
2005-04-29           

In [3]:
adjusted_close_columns = [col for col in combined_data.columns if col.endswith('_Adjusted Close')]

# Create a DataFrame to store the results
nan_zero_counts = pd.DataFrame(index=adjusted_close_columns, columns=['NaN Count', 'Zero Count'])

# Count NaN and zero values for each Adjusted Close column
for column in adjusted_close_columns:
    # Count NaN values
    nan_count = combined_data[column].isna().sum()
    # Count zero values (excluding NaN)
    zero_count = (combined_data[column] == 0).sum()
    
    # Store in results DataFrame
    nan_zero_counts.loc[column] = [nan_count, zero_count]

# Sort by NaN count in descending order
nan_zero_counts = nan_zero_counts.sort_values(by='NaN Count', ascending=False)

print(f"Found {len(adjusted_close_columns)} columns ending with '_Adjusted Close'")
print(nan_zero_counts)

Found 376 columns ending with '_Adjusted Close'
                    NaN Count Zero Count
CSCO_Adjusted Close         0          0
JPM_Adjusted Close          0          0
MCK_Adjusted Close          0          0
CNC_Adjusted Close          0          0
COP_Adjusted Close          0          0
...                       ...        ...
KO_Adjusted Close           0          0
IP_Adjusted Close           0          0
ADSK_Adjusted Close         0          0
ES_Adjusted Close           0          0
BLK_Adjusted Close          0          0

[376 rows x 2 columns]


In [4]:
def find_missing_data_tickers():
    # Identify tickers with missing data in adjusted close columns
    missing_data_tickers = []
    
    for column in adjusted_close_columns:
        ticker = column.replace('_Adjusted Close', '')
        # Check if there are any NaN values
        if combined_data[column].isna().any():
            missing_data_tickers.append(ticker)
    
    if missing_data_tickers:
        print(f"Found {len(missing_data_tickers)} tickers with missing data:")
        for ticker in missing_data_tickers:
            print(f"- {ticker}")
    else:
        print("No tickers with missing data found in the adjusted close columns.")

# Run the function
find_missing_data_tickers()

No tickers with missing data found in the adjusted close columns.


In [5]:
# Get list of tickers with missing data
missing_data_tickers = []
for column in adjusted_close_columns:
    ticker = column.replace('_Adjusted Close', '')
    if combined_data[column].isna().any():
        missing_data_tickers.append(ticker)

# Create a new DataFrame without columns from tickers with missing data
clean_data = combined_data.copy()

# Remove all columns related to problematic tickers
for ticker in missing_data_tickers:
    columns_to_drop = [col for col in clean_data.columns if col.startswith(f"{ticker}_")]
    clean_data = clean_data.drop(columns=columns_to_drop)

# Get the list of remaining tickers
remaining_tickers = list(set([col.split('_')[0] for col in clean_data.columns if '_' in col]))
print(f"Remaining tickers after cleaning: {len(remaining_tickers)}")

# Export only the list of tickers as CSV
tickers = [col.replace('_Adjusted Close','') for col in adjusted_close_columns]
import pandas as pd
pd.DataFrame({'ticker': tickers}).to_csv('tickers_list.csv', index=False)
print(f'Saved {len(tickers)} tickers to tickers_list.csv')

Remaining tickers after cleaning: 376
Saved 376 tickers to tickers_list.csv
Saved 376 tickers to tickers_list.csv


In [6]:
import os

# Save the clean dataframe to CSV
filename = 'clean_sp500_data.csv'
clean_data.to_csv(filename)
print(f"Data successfully saved to {filename}")

# Check file size
file_size_mb = os.path.getsize(filename) / (1024 * 1024)
print(f"File size: {file_size_mb:.2f} MB")

Data successfully saved to clean_sp500_data.csv
File size: 152.33 MB
